In [ ]:
import pandas as pd
import numpy as np
from numpy.random import uniform
from random import sample
from math import isnan

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('./OnlineRetail_RAW.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
import datetime as datetime
df = df.loc[df['Country']=='United Kingdom']
Recency = df.groupby(by ='CustomerID')['InvoiceDate'].max() - datetime.datetime.strptime('2010-01-12','%Y-%m-%d')
Recency = Recency.astype('timedelta64[D]') # timedelta64형식은 판다스가 아니므로 일별 데이터로 변환하기 위해 다음과 같이 사용 함 
Recency = Recency.reset_index()

In [ ]:
Recency = Recency.rename(columns = {'InvoiceNo' : 'Recency'})
Recency.head(3)

,CustomerID,InvoiceDate
0,12346.0,371.0
1,12747.0,674.0
2,12748.0,692.0


In [ ]:
# 기준 2개 동시적용하여 빈도세기 
df.groupby(['CustomerID','InvoiceNo']).count()

StockCode  Description  Quantity  InvoiceDate  \
CustomerID InvoiceNo                                                  
12346.0    541431             1            1         1            1   
           C541433            1            1         1            1   
12747.0    537215             7            7         7            7   
           538537             8            8         8            8   
           541677             5            5         5            5   
...                         ...          ...       ...          ...   
18283.0    579673            52           52        52           52   
           580872            50           50        50           50   
18287.0    554065            29           29        29           29   
           570715            38           38        38           38   
           573167             3            3         3            3   

                      UnitPrice  Country  
CustomerID InvoiceNo                      
12346.0    541431             1        1  
           C541433            1        1  
12747.0    537215             7        7  
           538537             8        8  
           541677             5        5  
...                         ...      ...  
18283.0    579673            52       52  
           580872            50       50  
18287.0    554065            29       29  
           570715            38       38  
           573167             3        3  

[19857 rows x 6 columns]

In [ ]:
# 기준 2개를 1차, 2차에 나누어 적용하여 빈도세기

# 1차적으로 고객별로 groupby한뒤 2차적으로 영수증을 기준으로 빈도 세기
Frequency = df.groupby(['CustomerID']).agg({'InvoiceNo' : ['count']})
Frequency = Frequency.stack().reset_index().drop('level_1',axis=1)
Frequency = Frequency.set_index('CustomerID')
Frequency = Frequency.reset_index()
Frequency = Frequency.rename(columns = {'InvoiceNo' : 'Frequency'})
Frequency.head(3)
# 다중 인덱스 해제, 재설정
# stack() or unstack( )을 이용해서, Series형 데이터를 DataFrame형의 데이터로 변경

,CustomerID,Frequency
0,12346.0,2
1,12747.0,103
2,12748.0,4642


In [ ]:
Amount = df.groupby(['CustomerID','InvoiceNo','Description'])['Quantity'].sum() * df.groupby(['CustomerID','InvoiceNo','Description'])['UnitPrice'].sum()
Amount = Amount.groupby('CustomerID').sum()
Amount = Amount.reset_index()
Amount = Amount.rename(columns = {0 : 'Amount'})
Amount.head(3)

,CustomerID,Amount
0,12346.0,0.00
1,12747.0,4196.01
2,12748.0,32034.17


In [ ]:
from functools import reduce
import pandas as pd
dfs = [Recency, Frequency, Amount]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='CustomerID'), dfs)
df_merge

,CustomerID,InvoiceDate,Frequency,Amount
0,12346.0,371.0,2,0.00
1,12747.0,674.0,103,4196.01
2,12748.0,692.0,4642,32034.17
3,12749.0,674.0,231,3868.20
4,12820.0,652.0,59,942.34
...,...,...,...,...
3945,18280.0,537.0,10,180.60
3946,18281.0,693.0,7,80.82
3947,18282.0,604.0,13,176.60
3948,18283.0,687.0,756,2329.67


In [ ]:
df_merge.to_csv('OnlineRetail_after.csv')